In [34]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import nilearn
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from nilearn.input_data import NiftiLabelsMasker, NiftiMasker, NiftiSpheresMasker
from nilearn import datasets
from nilearn.regions import RegionExtractor
from nilearn import plotting
import os
from util import *

In [10]:
atlas = datasets.fetch_atlas_juelich("maxprob-thr25-2mm")
masks = {"premotor":"GM Premotor cortex BA6", 
         "motor": "GM Primary motor cortex BA4a",
         "v1": "GM Visual cortex V1 BA17",
         "sup_parietal": "GM Superior parietal lobule 7P",
         "inf_parietal": "GM Inferior parietal lobule PF",
         "v5": "GM Visual cortex V5"
        }
subject_list = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '11', '12', '13', '14', '15', '16', '17', '18', '19']

In [19]:

for subject_id in subject_list:
    for mask in masks.keys():

        for run in range(1, 11):
            X = []
            run_no = str(run).zfill(2)
            beta_maps_dir = f'analyses_results/sub-{subject_id}/beta_maps/trial_level/run-{run_no}/'
            run_beta_maps = os.listdir(f'analyses_results/sub-{subject_id}/beta_maps/trial_level/run-{run_no}/')

            z_maps = {}

            for i in run_beta_maps:
                if 'prep' in i:
                    z_maps[i[5:-7]] = image.load_img(os.path.join(beta_maps_dir, i))

            #Sorted by first number
            all_images = image.concat_imgs((z_maps['153'], z_maps['207'], z_maps['45'], z_maps['99']))

            mask_image = image.new_img_like(atlas.maps, image.get_data(atlas.maps) == np.where(np.array(atlas.labels) == masks[mask])[0][0])
            resampled_mask = image.resample_to_img(mask_image, all_images, interpolation='nearest')

            fmri_masked = masking.apply_mask(all_images, mask_img=resampled_mask)
            X = fmri_masked
            y = np.tile(np.repeat(['153', '207', '45', '99'], 10), 1)
            #svm_predictions.append(cross_val_predict(OneVsRestClassifier(svm.SVC(probability = True)), X, y, cv=10, method='predict_proba'))
            dec_tree_predictions.append(cross_val_predict(tree.DecisionTreeClassifier(max_depth=4), X, y, cv=10, method='predict_proba'))
            knn_predictions.append(cross_val_predict(KNeighborsClassifier(), X, y, cv=10, method='predict_proba'))

        print("Mask done: ", mask)
    print('Subject done:', subject_id)



NameError: name 'dec_tree_predictions' is not defined

In [38]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(5096, 120)  # 5*5 from image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 4)

    def forward(self, x):
        x = torch.flatten(x, 1) # flatten all dimensions except the batch dimension
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [39]:
a = Net()

In [40]:
out = a(torch.from_numpy(fmri_masked))

In [59]:
target = torch.from_numpy(np.tile(np.repeat([0, 1, 2, 3], 10), 1).astype(int))
target_onehot = F.one_hot(target)


In [60]:
target_onehot.shape

torch.Size([40, 4])

In [63]:
criterion = nn.MSELoss()
loss = criterion(out, target)

RuntimeError: The size of tensor a (4) must match the size of tensor b (40) at non-singleton dimension 1